# 利用Qwen 翻译题目

In [1]:
import os
import time

import pandas as pd
from openai import OpenAI
import pandas

In [2]:
language_list = ["python","javascript","typescript","c&cpp"]
suffix_map = {
    "python": "py",
    "javascript": "js",
    "typescript": "ts",
    "java": "java",
    "c&cpp": "cpp"
}
client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key="sk-ff167cad89364e2491fc6193e006c4a3",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

In [ ]:
prompt_message_template = {
    # py 2 js
    "python_answer_to_javascript": "please change this python code to javascript code,the python code as below {python_code}",
    "python_signature_to_javascript": "please change this python code signature to javascript code signature,keep the code comments, the python code signature as below {python_code}",
    "python_test_to_javascript": "please change this python unittest test class to javascript testcase use jest descript, the python unittest test code as below {python_code}",
    # py 2 ts
    "python_answer_to_typescript": "please change this python code to typescript code,the python code  as below {python_code}",
    "python_signature_to_typescript": "please change this python code signature to typescript code signature,keep the code comments,  the python code signature as below {python_code}",
    "python_test_to_typescript": "please change this python unittest test class to typescript testcase use jest descript, the python unittest test code as below {python_code}",
    # py 2 c&cpp
    "python_answer_to_c&cpp": "please change this python code to c&cpp code,the python code  as below {python_code}",
    "python_signature_to_c&cpp": "please change this python code signature to cpp code signature,keep the code comments,  the python code signature as below {python_code}",
    "python_test_to_c&cpp": "please change this python unittest test class to cpp testcase use catch2, the python unittest test code as below {python_code}",
    # py 2 java
    "python_answer_to_java": "please change this python code to java code,package is org.real.temp,class is Answer, the python code  as below {python_code}",
    "python_signature_to_java": "please change this python code signature to java code signature,keep the code comments,  the python code signature as below {python_code}",
    "python_test_to_java": "please change this python unittest test class to java testcase use junit,package is org.real.temp,class is Tester,the python unittest test code as below {python_code}",

    # js 2 py
    "javascript_answer_to_python": "please change this javascript code to python code,the javascript code as below {javascript_code}",
    "javascript_signature_to_python": "please change this javascript code signature to python code signature,keep the code comments, the javascript code signature as below {javascript_code}",
    "javascript_test_to_python": "please change this javascript junit testcase to python testcase use unittest, the javascript jest testcase code as below {javascript_code}",
    # js 2 ts
    "javascript_answer_to_typescript": "please change this javascript code to typescript code,the javascript code as below {javascript_code}",
    "javascript_signature_to_typescript": "please change this javascript code signature to typescript code signature,keep the code comments, the javascript code signature as below {javascript_code}",
    "javascript_test_to_typescript": "please change this javascript jest testcase to typescript testcase use jest describe testcase, the javascript jest testcase code as below {javascript_code}",
    # js 2 c&cpp
    "javascript_answer_to_c&cpp": "please change this javascript code to cpp code,the javascript code as below {javascript_code}",
    "javascript_signature_to_c&cpp": "please change this javascript code signature to cpp code signature,keep the code comments, the javascript code signature as below {javascript_code}",
    "javascript_test_to_c&cpp": "please change this javascript jest testcase to cpp testcase use catch2 testcase,the javascript jest testcase code as below {javascript}",
    # js 2 java
    "javascript_answer_to_java": "please change this javascript code to java code,the javascript code as below {javascript_code}",
    "javascript_signature_to_java": "please change this javascript code signature to java code signature,keep the code comments, the javascript code signature as below {javascript_code}",
    "javascript_test_to_java": "please change this javascript jest testcase to java testcase use junit,package is org.real.temp,class is Tester,the javascript jest testcode as below {javascript_code}"

}

In [4]:
from pathlib import Path

for language in language_list:
    excel_data = pd.read_excel("./xlsx/RealisticEval-Data.xlsx", sheet_name=language)
    data = excel_data[excel_data['check'] == 'yes']
    l_list = [x for x in ["python", "javascript", "typescript", "c&cpp", "java"] if x != language]
    for index, row in data.iterrows():
        task_id = row['task_id']
        if task_id in [i for i in range(1, 266)]:
            continue
        work_path = f"../all/t{task_id}"
        # 读取signature代码
        signature_path = f"{work_path}/{language}/signature.{suffix_map[language]}"
        with open(signature_path, "r", encoding="utf8") as file:
            signature_code = file.read()
        # 读取test代码
        test_path = f"{work_path}/{language}/test.{suffix_map[language]}"
        with open(signature_path, "r", encoding="utf8") as file:
            test_code = file.read()
        for l in l_list:
                        # 创建文件夹
            if l != "java":
                Path(f"{work_path}/{l}").mkdir(parents=True,exist_ok=True)
                answer_file_path = Path(f"{work_path}/{l}/answer.{suffix_map[l]}")
                signature_file_path = Path(f"{work_path}/{l}/signature.{suffix_map[l]}")
                test_file_path = Path(f"{work_path}/{l}/test.{suffix_map[l]}")
                if not answer_file_path.exists():
                    answer_file_path.touch()
                if not signature_file_path.exists():
                    signature_file_path.touch()
                if not test_file_path.exists():
                    test_file_path.touch()
            elif l == "java":
                Path(f"{work_path}/{l}").mkdir(parents=True,exist_ok=True)
                answer_file_path = Path(f"{work_path}/{l}/Answer.{suffix_map[l]}")
                signature_file_path = Path(f"{work_path}/{l}/Signature.{suffix_map[l]}")
                test_file_path = Path(f"{work_path}/{l}/Tester.{suffix_map[l]}")
                if not answer_file_path.exists():
                    answer_file_path.touch()
                if not signature_file_path.exists():
                    signature_file_path.touch()
                if not test_file_path.exists():
                    test_file_path.touch()
            print(f"{language}_{task_id}_to_{l} ===============================\n")
            # 签名转换=============================
            signature_key = f"{language}_signature_to_{l}"
            signature_template = prompt_message_template[signature_key].format(python_code=signature_code)
            completion = client.chat.completions.create(
                model="qwen-coder-turbo",  # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
                messages=[
                    {'role': 'system', 'content': 'You are a helpful assistant.'},
                    {'role': 'user', 'content': signature_template}],
            )
            llm_signature_response = completion.choices[0].message.content
            with open(f"{work_path}/{l}/signature.txt", "w", encoding="utf8") as file:
                file.write(llm_signature_response)
                file.flush()

            # 测试转换=============================
            test_key = f"{language}_test_to_{l}"
            test_template = prompt_message_template[test_key].format(python_code=test_code)
            completion = client.chat.completions.create(
                model="qwen-coder-turbo",  # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
                messages=[
                    {'role': 'system', 'content': 'You are a helpful assistant.'},
                    {'role': 'user', 'content': test_template}],
            )
            llm_test_response = completion.choices[0].message.content
            with open(f"{work_path}/{l}/test.txt", "w", encoding="utf8") as file:
                file.write(llm_test_response)
                file.flush()

            # 答案转换 =============================
            answer_key = f"{language}_answer_to_{l}"
            answer_template = prompt_message_template[answer_key].format(python_code=test_code)
            completion = client.chat.completions.create(
                model="qwen-coder-turbo",  # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
                messages=[
                    {'role': 'system', 'content': 'You are a helpful assistant.'},
                    {'role': 'user', 'content': answer_template}],
            )
            print(completion)
            llm_answer_response = completion.choices[0].message.content
            with open(f"{work_path}/{l}/answer.txt", "w", encoding="utf8") as file:
                file.write(llm_answer_response)
                file.flush()
            time.sleep(0.5)


python_266_to_javascript ===============================

ChatCompletion(id='chatcmpl-de358c25-ad34-9d69-8a41-ad8643ab54f7', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Here is the JavaScript version of your Python code:\n\n```javascript\nfunction handleNestedData(data) {\n    /**\n     * Handle nested question structures (e.g., objects, arrays, and enums),\n     * decode bytes to UTF8-strings, and convert numbers to integers or floating point numbers.\n     *\n     * @param {Object} data - Question object\n     * @return {Object} After converted question\n     */\n\n    if (typeof data === 'object' && data !== null) {\n        for (let key in data) {\n            if (data.hasOwnProperty(key)) {\n                let value = data[key];\n\n                // If value is an array, recursively call handleNestedData on each item\n                if (Array.isArray(value)) {\n                    data[key] = value.map(item => handleNeste

KeyError: 'javascript_signature_to_python'